In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

FBM_LSTM不如LSTM好用，LSTM/GRU 是局部建模器，只适合短期记忆，我打算用FBM-NP，更适合长期预测

我打算用FBM-NL + 多步收益率预测

In [29]:
#配置参数
T = 128  # 每个滑动窗口的时间序列长度（即模型输入序列长度）
L = 1    # 预测步数（例如预测下一个 close 价格）
stride = 1 #滑动窗口的步长
num_epochs = 50 # 模型训练的轮数
batch_size = 32 # 每一批训练样本的数量

# 自动选择设备（如果有GPU就用GPU，否则用CPU）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
# 数据读取与滑窗
df = pd.read_csv('SP500.csv', parse_dates=['date'])
df.sort_values('date', inplace=True)
close = df['close'].values
returns = np.diff(np.log(close))  # 对数的收益率

# 检查输入长度T是否是偶数
if T % 2 != 0:
    T += 1

# 生成滑窗和目标
data = np.lib.stride_tricks.sliding_window_view(returns, window_shape=T)[::stride]
y = returns[T + (L - 1):]  # 预测L步后的值
X_raw = data[:len(y)]  # 适配X和y的长度
X_raw = (X_raw - X_raw.mean()) / (X_raw.std() + 1e-8)  # 原始输入平稳化

In [31]:
# 通用 Dataset 类
#将输入特征 X 和目标标签 y 封装为 PyTorch 可处理的数据集，好进行一些操作
class SeqDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)#将输入特征转换为 float32 类型的张量
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)#在最后一维添加维度
    def __len__(self):#返回数据集的样本数量
        return len(self.X)
    def __getitem__(self, idx):#根据索引 idx 返回对应的特征和标签样本
        return self.X[idx], self.y[idx]

In [ ]:
# FBM特征生成函数
#输入长度为T的一维数组，输出二维数组
def fbm_features(x):
    T = len(x)
    H = np.fft.fft(x) # 傅里叶变换
    HR, HI = H.real, H.imag
    K = T // 2 + 1
    N = np.arange(T).reshape(-1, 1)# 时间索引列向量 (T, 1)
    ks = np.arange(K).reshape(1, -1)# 频率索引行向量 (1, K)

    # 生成正交基函数矩阵 (文献公式5和6)
    C = np.cos(2 * np.pi * N @ ks / T)/T
    S = -np.sin(2 * np.pi * N @ ks / T)/T

    #初始化频域系数 (文献公式7和8)
    ak = np.zeros(K)
    bk = np.zeros(K)
    ak[0] = HR[0]          # 直流分量
    bk[0] = 0              # 直流分量无虚部
    ak[-1] = HR[T//2]      # 奈奎斯特频率分量
    bk[-1] = 0             # 奈奎斯特频率分量无虚部
    ak[1:-1] = 2 * HR[1:T//2]  # 中间频率分量实部缩放
    bk[1:-1] = 2 * HI[1:T//2]  # 中间频率分量虚部缩放
    
    fbm_feature = np.concatenate([C * ak, S * bk], axis=1)
    fbm_feature = (fbm_feature - fbm_feature.mean()) / (fbm_feature.std() + 1e-8)
    return fbm_feature

In [33]:
fbm_tensor = np.stack([fbm_features(x) for x in X_raw], axis=0)
X_fbm = fbm_tensor[:len(y)]
X_raw = X_raw[:, :, np.newaxis]  # 输入调整维度

In [34]:
# 顺序分割数据，防止时间混淆
train_size = int(0.8 * len(X_raw))
train_raw = SeqDataset(X_raw[:train_size], y[:train_size])
val_raw = SeqDataset(X_raw[train_size:], y[train_size:])
train_fbm = SeqDataset(X_fbm[:train_size], y[:train_size])
val_fbm = SeqDataset(X_fbm[train_size:], y[train_size:])

train_loader_raw = DataLoader(train_raw, batch_size=batch_size, shuffle=True)
val_loader_raw = DataLoader(val_raw, batch_size=batch_size)
train_loader_fbm = DataLoader(train_fbm, batch_size=batch_size, shuffle=True)
val_loader_fbm = DataLoader(val_fbm, batch_size=batch_size)

In [35]:
# LSTM 网络结构
#LSTMModel 是一个继承自 PyTorch nn.Module 的神经网络模型：LSTM层（提取序列中的时序特征）+全连接层（将 LSTM 的输出映射到最终预测值）
class FBM_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.norm = nn.LayerNorm(hidden_dim)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, 1)
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.norm(out[:, -1, :])
        return self.mlp(out)

In [36]:
# 验证函数
def evaluate_model(model, dataloader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for x_batch, y_batch in dataloader:
            x_batch = x_batch.to(device)
            pred = model(x_batch).cpu().numpy()
            y_true.append(y_batch.numpy())
            y_pred.append(pred)
    y_true = np.concatenate(y_true).flatten()
    y_pred = np.concatenate(y_pred).flatten()
    return mean_absolute_error(y_true, y_pred), np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
# 训练函数
def train_model(model, train_loader, val_loader,patience=5):
    model.to(device)#将模型移至指定设备（CPU/GPU）
    #Adam 优化器：自适应学习率优化算法
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience//2)
    criterion = nn.MSELoss()#均方误差损失函数
    best_val_mae = float('inf')
    early_stop_counter = 0

    for epoch in range(1, num_epochs + 1):
        model.train()#设置训练模式：启用 Dropout 等训练专用层
        total_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()#防止梯度累积
            loss = criterion(model(x_batch), y_batch)
            loss.backward()#反向传播,计算梯度
            optimizer.step()#更新模型权重
            total_loss += loss.item()#记录本轮总损失
        

        # 验证阶段
        val_mae, val_rmse = evaluate_model(model, val_loader)
        scheduler.step(val_mae)

        
        # 早停机制
        if val_mae < best_val_mae:
            best_val_mae = val_mae
            best_model = model.state_dict()
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
        
        print(f"Epoch {epoch:02d}: TrainLoss={total_loss/len(train_loader):.4f}, Val MAE={val_mae:.4f}, RMSE={val_rmse:.4f}")

    
    # 加载最佳模型
    model.load_state_dict(best_model)
    return model, best_val_mae

In [38]:
# 训练 baseline LSTM（原始序列）:定义模型、训练模型、评估模型
print("\n🔹 Training baseline LSTM...")
model_raw = FBM_LSTM(input_dim=1)  # 保持同一结构
model_raw, _ = train_model(model_raw, train_loader_raw, val_loader_raw)
mae_raw, rmse_raw = evaluate_model(model_raw, val_loader_raw)


🔹 Training baseline LSTM...
Epoch 01: TrainLoss=0.0126, Val MAE=0.0066, RMSE=0.0122
Epoch 02: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0123
Epoch 03: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0121
Epoch 04: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0121
Epoch 05: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0121
Epoch 06: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0121
Epoch 07: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0121
Epoch 08: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0120
Epoch 09: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0120
Epoch 10: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0120
Epoch 11: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0121
Epoch 12: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0121
Epoch 13: TrainLoss=0.0002, Val MAE=0.0066, RMSE=0.0121
Early stopping at epoch 14


In [39]:
# 训练 FBM-LSTM（FBM特征）
print("\n🔹 Training FBM-LSTM...")
model_fbm = FBM_LSTM(input_dim=X_fbm.shape[2])
model_fbm, _ = train_model(model_fbm, train_loader_fbm, val_loader_fbm)
mae_fbm, rmse_fbm = evaluate_model(model_fbm, val_loader_fbm)


🔹 Training FBM-LSTM...
Epoch 01: TrainLoss=0.0088, Val MAE=0.0091, RMSE=0.0136
Epoch 02: TrainLoss=0.0003, Val MAE=0.0102, RMSE=0.0142
Epoch 03: TrainLoss=0.0003, Val MAE=0.0105, RMSE=0.0144
Epoch 04: TrainLoss=0.0002, Val MAE=0.0080, RMSE=0.0131
Epoch 05: TrainLoss=0.0002, Val MAE=0.0070, RMSE=0.0123
Epoch 06: TrainLoss=0.0002, Val MAE=0.0071, RMSE=0.0123
Epoch 07: TrainLoss=0.0002, Val MAE=0.0084, RMSE=0.0130
Epoch 08: TrainLoss=0.0002, Val MAE=0.0068, RMSE=0.0122
Epoch 09: TrainLoss=0.0002, Val MAE=0.0082, RMSE=0.0128
Epoch 10: TrainLoss=0.0002, Val MAE=0.0067, RMSE=0.0122
Epoch 11: TrainLoss=0.0002, Val MAE=0.0069, RMSE=0.0123
Epoch 12: TrainLoss=0.0002, Val MAE=0.0067, RMSE=0.0120
Epoch 13: TrainLoss=0.0001, Val MAE=0.0070, RMSE=0.0128
Epoch 14: TrainLoss=0.0001, Val MAE=0.0067, RMSE=0.0120
Epoch 15: TrainLoss=0.0001, Val MAE=0.0068, RMSE=0.0121
Epoch 16: TrainLoss=0.0001, Val MAE=0.0069, RMSE=0.0123
Early stopping at epoch 17


In [ ]:
# 输出最终对比结果
print("\n✅ Final Evaluation:")
print(f"LSTM       → MAE: {mae_raw:.4f}, RMSE: {rmse_raw:.4f}")
print(f"FBM-LSTM   → MAE: {mae_fbm:.4f}, RMSE: {rmse_fbm:.4f}")


✅ Final Evaluation:
LSTM       → MAE: 0.0066, RMSE: 0.0121
FBM-LSTM   → MAE: 0.0068, RMSE: 0.0123
